# FS Consulting

## Load Model & New Production Data

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import model_1

# # Import new data for predictions & create empty DataFrame with shape[test_set.shape[0],1]
# production_data = pd.read_csv('./data/1814_Preprocessed_TestSet.csv')
# production_targets = pd.DataFrame(index=range(production_data.shape[0]),columns=range(1),dtype='float')

# Import new data for predictions & create empty DataFrame with shape[test_set.shape[0],1]
production_data = pd.read_csv('./data/TestSet.csv')
production_targets = pd.DataFrame(index=range(production_data.shape[0]),columns=range(1),dtype='float')

if isinstance(production_data, pd.DataFrame):
    print("New data successfully imported.")
else:
    print("New data failed to import.")
    
# # Load model and all of its trained parameters
# checkpoint = torch.load('checkpoint_v1.pth')
# model = model_1.JobPerformance(checkpoint['input_size'],checkpoint['output_size'],checkpoint['hidden_layers'],checkpoint['drop_out'])
# model.load_state_dict(checkpoint['state_dict'])

# print("Our Model: \n",model,"\n")

C:\Users\Jonathan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (50,255,256,257,258,260,268) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


New data successfully imported.


## Preprocess Data & Setup Data Loader

In [11]:
import pipeline_functions
from sklearn.externals import joblib

# Import preprocessing pipeline
preprocessing_pipeline = joblib.load('../2309_preprocessing_params.joblib') 

# Send production data through preprocessing
production_data_processed = preprocessing_pipeline.transform(production_data)

# Load preprocessed data into data loader
production_data = torch.utils.data.TensorDataset(torch.Tensor(np.array(production_data_processed)), torch.Tensor(np.array(production_targets)))
production_loader = torch.utils.data.DataLoader(production_data_processed, batch_size = 32, shuffle = False)

if(production_loader):
    print("New data successfully preprocessed and is ready for predictions.")
else:
    print("Preprocessing failed.")


Starting Shape: (16000, 378) 
  Forcing column 'v71' to numerical data. 
  Preprocessing: 
    Numerical: Missing Value Ratio Filter (>0.5) 
      Starting Numerical Features:  84
      Remaining Numerical Features: 36
    Numerical: Imputation 
    Numerical: Normalization 
    Numerical: Low Variance Filter (>0.01) 
      Starting Numerical Features:  36
      Remaining Numerical Features: 30
    Categorical: Missing Value Ratio Filter (>0.5) 
      Starting Categorical Features:  294
      Remaining Categorical Features: 294
    Categorical: Imputation 
    Categorical: Conversion of Ints to Strings 
    Categorical: One Hot Encoding 
      Starting Categorical Features: 294
      Remaining Categorical Features: 2769
    Categorical: Low Variance Filter (>0.01) 
      Starting Categorical Features:  2769
      Remaining Categorical Features: 2769
  Recombined Numerical & Categorical Shape:  (20000, 2799)
  Dimensionality Reduction:  
    Converted Matrix to DataFrame
    High Corre

## Run Model & Save Predictions
Expected Results: 31,000 MSE

In [18]:
predictions_tensor = model_1.predict(model, production_loader)
output_dataframe = pd.DataFrame(predictions_tensor.detach().numpy().round(2))

raw_test_set = pd.read_csv('./data/TestSet.csv')
raw_test_set['job_performance'] = output_dataframe.iloc[:,0]
raw_test_set.to_csv(r'./data/predictions_v2.csv', index=False)
print("Predictions data saved to CSV file.")

Predictions data saved to CSV file.
